# Web Scraping pour l'obtention du nom et de la position des capitales


In [2]:
!pip install selenium

     |████████████████████████████████| 904 kB 9.0 MB/s 


In [3]:
!apt install chromium-chromedriver

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following additional packages will be installed:
  chromium-browser chromium-browser-l10n chromium-codecs-ffmpeg-extra
Suggested packages:
  webaccounts-chromium-extension unity-chromium-extension
The following NEW packages will be installed:
  chromium-browser chromium-browser-l10n chromium-chromedriver
  chromium-codecs-ffmpeg-extra
0 upgraded, 4 newly installed, 0 to remove and 37 not upgraded.
Need to get 91.8 MB of archives.
After this operation, 315 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bionic-updates/universe amd64 chromium-codecs-ffmpeg-extra amd64 92.0.4515.159-0ubuntu0.18.04.1 [1,124 kB]
Get:2 http://archive.ubuntu.com/ubuntu bionic-updates/universe amd64 chromium-browser amd64 92.0.4515.159-0ubuntu0.18.04.1 [81.7 MB]
Get:3 http://archive.ubuntu.com/ubuntu bionic-updates/universe amd64 chromium-browser-l10n all 92.0.4515.159-0ubuntu0.1

In [4]:
pip install webdriver-manager

In [5]:
# Import des bibliothèques
import requests
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.firefox.firefox_binary import FirefoxBinary

from pathlib import *
import time
import sys

In [6]:
# URL de départ : page wikipédia des capitales
url_capital_all = 'https://fr.wikipedia.org/wiki/Liste_des_capitales_du_monde'

res = requests.get(url_capital_all)
print(res) # <Response [200]> : Tout va bien !

<Response [200]>


In [7]:
soup = BeautifulSoup(res.content, 'html.parser')
print(soup.title) # Affichage du titre de la page wikipédia des capitales

<title>Liste des capitales du monde — Wikipédia</title>


In [8]:
# Obtention de la liste des capitales
data = []
data_url = []

table = soup.find('table', attrs={'class','wikitable sortable alternance'})

for indice, tr in enumerate(table.find('tbody').find_all('tr')):
  if indice != 0:
    td = tr.find_all('td')
    for indice_td, elt in enumerate(td):
      if indice_td != 0:
        result = elt.find('a')
        if result != None:
          data.append(result.get_text())
          data_url.append(str(result.get_attribute_list).split('=')[2].split('>')[0].replace(' ', '_'))

print(data) # Liste des capitales
print(data_url) # URL correspondantes

['Abou Dabi', 'Abuja', 'Accra', 'Achgabat', 'Addis-Abeba', 'Alger', 'Alofi', 'Amman', 'Amsterdam', 'Andorre-la-Vieille', 'Ankara', 'Antananarivo', 'Apia', 'Asmara', 'Asuncion', 'Athènes', 'Avarua', 'Bagdad', 'Bakou', 'Bamako', 'Bandar Seri Begawan', 'Bangkok', 'Bangui', 'Banjul', 'Basseterre', 'Belgrade', 'Belmopan', 'Berlin', 'Berne', 'Beyrouth', 'Bichkek', 'Bissau', 'Bloemfontein', 'Bogota', 'Brasilia', 'Bratislava', 'Brazzaville', 'Bridgetown', 'Bruxelles', 'Bucarest', 'Budapest', 'Buenos Aires', 'Le Caire', 'Canberra', 'Le Cap', 'Caracas', 'Castries', 'Chisinau', 'Sri Jayawardenapura Kotte', 'Conakry', 'Copenhague', 'Dakar', 'Damas', 'Delap-Uliga-Darrit', 'Dacca', 'Dili', 'Djibouti', 'Djouba', 'Dodoma', 'Doha', 'Douchanbé', 'Dublin', 'Erevan', 'Freetown', 'Funafuti', 'Gaborone', 'Georgetown', 'Gitega', 'Guatemala', 'Hanoï', 'Harare', 'La Havane', 'Helsinki', 'Honiara', 'Islamabad', 'Jakarta', 'Jérusalem', 'Jérusalem-Est', 'Kaboul', 'Kampala', 'Katmandou', 'Khartoum', 'Kiev', 'Kigal

In [9]:
# Degrees Minutes Seconds --> Decimal Degrees
def dms_to_dd(d, m, s):
    dd = d + m/60 + s/3600
    return dd

In [10]:
from selenium import webdriver
from pathlib import * 

# path = Path('/content/geckodriver.exe')
# driver = webdriver.Firefox(executable_path=path)

In [11]:
# Quelques imports
import sys
sys.path.insert(0,'/content/geckodriver')
from selenium import webdriver
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')
wd = webdriver.Chrome('chromedriver',chrome_options=chrome_options)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:8: DeprecationWarning: use options instead of chrome_options
  


In [12]:
# Obtention des coordonnées à partir de la page wikipédia de la capitale
def get_gps(capital):
  url = f'https://fr.wikipedia.org/wiki/{capital}#/maplink/0'
  
  wd.get(url)

  time.sleep(1/2)

  try:
    capital = wd.find_element_by_css_selector('.mw-kartographer-captionfoot')
    text =  capital.text
  except:
    try:
      capital = wd.find_element_by_xpath('/html/body/div[7]/div/div[1]/div[2]/div[3]/div/div[2]')
      text =  capital.text
    except:
      text =  ''
   
  print(capital, text) # Affichage pour vérification
  return text

In [13]:
# Extraction des coordonnées à partir de la chaîne de caractères
def gps_to_pos(gps_param):
  if gps_param == '':
    return [None, None]
  else:
    liste = gps_param.replace('°', '').replace('′', '').replace('″', '').replace(', ',',')
    liste = liste.split(',')
    lat = liste[0].split(' ')
    lon = liste[1].split(' ')

    if isinstance(lat[-1], str):
      lat[-1] = 0
    if isinstance(lon[-1], str):
      lon[-1] = 0

    # lattitude = dms_to_dd(float(lat[0]), float(lat[1]), float(lat[2]))
    # longitude = dms_to_dd(float(lon[0]), float(lon[1]), float(lon[2]))
    return [dms_to_dd(float(lat[0]), float(lat[1]), float(lat[2])), dms_to_dd(float(lon[0]), float(lon[1]), float(lon[2]))]

In [14]:
liste_latlong = []
for capital in data_url:
  print(capital) # Nom de la capitale dans l'url
  gps = get_gps(capital.replace('"', ''))
  liste_latlong.append(gps_to_pos(gps))

"Abou_Dabi_(ville)"
<selenium.webdriver.remote.webelement.WebElement (session="2f617ff2ad25b3f03a76f8f9de79fa4c", element="4daf5700-ba9e-4164-b79b-b15ad85d071b")> 24° 27′ 54″ nord, 54° 21′ 57″ est
"Abuja"
<selenium.webdriver.remote.webelement.WebElement (session="2f617ff2ad25b3f03a76f8f9de79fa4c", element="012059f8-26c8-45fb-8959-88ba9e08d2b7")> 9° 03′ 20″ nord, 7° 29′ 29″ est
"Accra"
<selenium.webdriver.remote.webelement.WebElement (session="2f617ff2ad25b3f03a76f8f9de79fa4c", element="f9d280ef-87c6-4c91-b0be-c48cfc64355d")> 5° 33′ 29″ nord, 0° 12′ 04″ ouest
"Achgabat"
<selenium.webdriver.remote.webelement.WebElement (session="2f617ff2ad25b3f03a76f8f9de79fa4c", element="d8022fcc-2951-454d-8b8d-af04a369fefd")> 37° 57′ 00″ nord, 58° 23′ 00″ est
"Addis-Abeba"
<selenium.webdriver.remote.webelement.WebElement (session="2f617ff2ad25b3f03a76f8f9de79fa4c", element="773e1f71-e3f4-4f2d-8337-fcef5d446904")> 9° 02′ nord, 38° 44′ est
"Alger"
<selenium.webdriver.remote.webelement.WebElement (session

In [15]:
print(len(liste_latlong), len(data_url)) 
# Vérification que les deux listes ont la même longueur de liste : pas d'erreur pour une des capitales


# Erreur pour des capitales comme Paris --> pas de coordonnées trouvées

202 202


In [16]:
# Création de la carte des capitales via Folium
import folium
c= folium.Map(location=[43.634, 1.433333])

for coord, capital in zip(liste_latlong, data):
    try:
      folium.Marker([coord[0], coord[1]], popup=capital).add_to(c)
    except:
      continue

c # Affichage de la carte